In [1]:
# module import
import pystan
import numpy as np
import pylab as py
import pandas as pd
 
## data simulation
x = np.arange(1, 100, 5)
y = 2.5 + .5 * x + np.random.randn(20) * 10
 
# get number of observations
N = len(x)
 
# plot the data
py.plot(x,y, 'o')
py.show()
 
# STAN model (this is the most important part)
regress_code = """
data {
 int<lower = 0> N; // number of observations
 real y[N]; // response variable
 real x[N]; // predictor variable
}
parameters {
 real a; // intercept
 real b; // slope
 real<lower=0> sigma; // standard deviation
}
transformed parameters {
 real mu[N]; // fitted values
 
for(i in 1:N)
 mu[i] <- a + b*x[i];
}
model {
 y ~ normal(mu, sigma);
}
"""
 
# make a dictionary containing all data to be passed to STAN
regress_dat = {'x': x,
 'y': y,
 'N': N}
 
# Fit the model
fit = pystan.stan(model_code=regress_code, data=regress_dat,
 iter=1000, chains=4)
 
# model summary
print fit
 
# show a traceplot of ALL parameters. This is a bear if you have many
fit.traceplot()
py.show()
 
# Instead, show a traceplot for single parameter
fit.plot(['a'])
py.show()
 
##### PREDICTION ####
 
# make a dataframe of parameter estimates for all chains
params = pd.DataFrame({'a': fit.extract('a', permuted=True), 'b': fit.extract('b', permuted=True)})
 
# next, make a prediction function. Making a function makes every step following this 10 times easier
def stanPred(p):
 fitted = p[0] + p[1] * predX
 return pd.Series({'fitted': fitted})
 
# make a prediction vector (the values of X for which you want to predict)
predX = np.arange(1, 100)
 
# get the median parameter estimates
medParam = params.median()
# predict
yhat = stanPred(medParam)
 
# get the predicted values for each chain. This is super convenient in pandas because
# it is possible to have a single column where each element is a list
chainPreds = params.apply(stanPred, axis = 1)
 
## PLOTTING
 
# create a random index for chain sampling
idx = np.random.choice(1999, 50)
# plot each chain. chainPreds.iloc[i, 0] gets predicted values from the ith set of parameter estimates
for i in range(len(idx)):
 py.plot(predX, chainPreds.iloc[idx[i], 0], color='lightgrey')
 
# original data
py.plot(x, y, 'ko')
# fitted values
py.plot(predX, yhat['fitted'], 'k')
 
# supplementals
py.xlabel('X')
py.ylabel('Y')
 
py.show()

Inference for Stan model: anon_model_f6ebce9f03467a5ca70e280e6b25c124.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

         mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a        6.46    0.24   4.57  -2.77   3.77   6.34   9.19  15.75  358.0    1.0
b        0.48  4.7e-3   0.08    0.3   0.43   0.48   0.53   0.64  299.0   1.01
sigma   10.24    0.09   1.85    7.6   8.93   9.97  11.25  14.75  381.0    1.0
mu[0]    6.94    0.24    4.5   -2.1   4.29   6.82   9.63  16.09  359.0    1.0
mu[1]    9.32    0.22   4.15   1.01   6.79   9.21  11.83  17.78  365.0    1.0
mu[2]    11.7     0.2   3.82    4.1   9.38   11.6  14.04  19.39  373.0    1.0
mu[3]   14.08    0.18    3.5   7.09  11.93   14.0  16.24   21.2  385.0    1.0
mu[4]   16.46    0.16   3.21  10.16  14.44  16.35   18.5  23.07  401.0    1.0
mu[5]   18.84    0.14   2.94  12.95  16.97  18.75  20.71  24.86  424.0    1.0
mu[6]   21.22    0.13    2.7  15.

IndexError: index out of bounds